In [1]:
import sys
sys.path.append('../')
path_data = '/home/Housing_prices_course/data/'

In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_squared_error, mean_squared_log_error, mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error, make_scorer
from sklearn.model_selection import cross_val_score, cross_validate

from sklearn.preprocessing import PolynomialFeatures

from sklearn.preprocessing import StandardScaler

from scipy.stats import loguniform
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

import warnings
import importlib
warnings.filterwarnings('ignore')
%matplotlib inline

In [3]:
def type_and_nulls(df):
    df_type = pd.concat([pd.DataFrame(df.dtypes), df.isnull().sum().to_frame().rename(columns={0:'nulls'})], axis=1)
    display(df_type.head())
    display(df.dtypes.value_counts())
    not_nulls = list((df_type[df_type['nulls']==0]).index)
    nulls = list((df_type[df_type['nulls']>0]).index)
    
    dict_type_columns = {}
    for type_column in list(df.dtypes.value_counts().index):
        dict_type_columns[str(type_column)] = list(df_type[df_type[0] == type_column].index)
    
    print('Nulls features: ', len(nulls))
    print('Not null features: ', len(not_nulls))
    
    dict_type_columns['nulls'] = nulls
    dict_type_columns['not_nulls'] = not_nulls
    
    
    return dict_type_columns, df_type


def correlation_heatmap(df, columns, figsize=(12, 9), plot=False):
    corrmat = df[columns].corr()
    sorted_columns = corrmat.sum(axis=1).sort_values().index.values
    corrmat = corrmat.loc[sorted_columns][sorted_columns]
    if plot:
        f, ax = plt.subplots(figsize=figsize)
        sns.heatmap(corrmat, vmax=.8, square=True)
    return corrmat

def polynomial_transformations(df_train, columns, degree=2, scale_method=None):
    if scale_method is not None:
        df_train, df_val = apply_scale(df_train[columns], df_val[columns], scale_method)
    poly_trans = PolynomialFeatures(degree=degree)
    df_result_train = pd.DataFrame(poly_trans.fit_transform(df_train[columns]))
        
    df_result_train.columns = poly_trans.get_feature_names_out()
    
    return df_result_train


In [4]:
df_train = pd.read_csv(path_data+'train.csv', index_col='Id')
df_test = pd.read_csv(path_data+'test.csv', index_col='Id')

In [5]:
dict_type_columns, df_type = type_and_nulls(df_train)

,0,nulls
MSSubClass,int64,0
MSZoning,object,0
LotFrontage,float64,259
LotArea,int64,0
Street,object,0


object     43
int64      34
float64     3
dtype: int64

Nulls features:  19
Not null features:  61


In [6]:
dict_type_columns.keys()

dict_keys(['object', 'int64', 'float64', 'nulls', 'not_nulls'])

In [7]:
numerical_features_not_null = set(dict_type_columns['int64'] + dict_type_columns['float64']) - set(dict_type_columns['nulls'])
numerical_features_not_null = list(numerical_features_not_null)
target = numerical_features_not_null.pop(numerical_features_not_null.index('SalePrice'))
len(numerical_features_not_null)
print(target)
print(len(numerical_features_not_null), numerical_features_not_null)

SalePrice
33 ['OverallCond', 'FullBath', 'YearRemodAdd', 'ScreenPorch', 'BsmtFinSF2', 'LotArea', 'BsmtFinSF1', 'WoodDeckSF', 'GarageArea', 'Fireplaces', 'LowQualFinSF', 'GarageCars', 'PoolArea', 'KitchenAbvGr', 'HalfBath', 'MSSubClass', 'BsmtHalfBath', 'MiscVal', 'OpenPorchSF', 'YearBuilt', 'OverallQual', '2ndFlrSF', '3SsnPorch', '1stFlrSF', 'TotalBsmtSF', 'BsmtUnfSF', 'BsmtFullBath', 'YrSold', 'MoSold', 'BedroomAbvGr', 'TotRmsAbvGrd', 'EnclosedPorch', 'GrLivArea']


In [8]:
df_train[numerical_features_not_null].describe()

,OverallCond,FullBath,YearRemodAdd,ScreenPorch,BsmtFinSF2,LotArea,BsmtFinSF1,WoodDeckSF,GarageArea,Fireplaces,...,1stFlrSF,TotalBsmtSF,BsmtUnfSF,BsmtFullBath,YrSold,MoSold,BedroomAbvGr,TotRmsAbvGrd,EnclosedPorch,GrLivArea
count,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,5.575342,1.565068,1984.865753,15.060959,46.549315,10516.828082,443.639726,94.244521,472.980137,0.613014,...,1162.626712,1057.429452,567.240411,0.425342,2007.815753,6.321918,2.866438,6.517808,21.954110,1515.463699
std,1.112799,0.550916,20.645407,55.757415,161.319273,9981.264932,456.098091,125.338794,213.804841,0.644666,...,386.587738,438.705324,441.866955,0.518911,1.328095,2.703626,0.815778,1.625393,61.119149,525.480383
min,1.000000,0.000000,1950.000000,0.000000,0.000000,1300.000000,0.000000,0.000000,0.000000,0.000000,...,334.000000,0.000000,0.000000,0.000000,2006.000000,1.000000,0.000000,2.000000,0.000000,334.000000
25%,5.000000,1.000000,1967.000000,0.000000,0.000000,7553.500000,0.000000,0.000000,334.500000,0.000000,...,882.000000,795.750000,223.000000,0.000000,2007.000000,5.000000,2.000000,5.000000,0.000000,1129.500000
50%,5.000000,2.000000,1994.000000,0.000000,0.000000,9478.500000,383.500000,0.000000,480.000000,1.000000,...,1087.000000,991.500000,477.500000,0.000000,2008.000000,6.000000,3.000000,6.000000,0.000000,1464.000000
75%,6.000000,2.000000,2004.000000,0.000000,0.000000,11601.500000,712.250000,168.000000,576.000000,1.000000,...,1391.250000,1298.250000,808.000000,1.000000,2009.000000,8.000000,3.000000,7.000000,0.000000,1776.750000
max,9.000000,3.000000,2010.000000,480.000000,1474.000000,215245.000000,5644.000000,857.000000,1418.000000,3.000000,...,4692.000000,6110.000000,2336.000000,3.000000,2010.000000,12.000000,8.000000,14.000000,552.000000,5642.000000


In [9]:
## Split training and test datasets
target = 'SalePrice'

X_train, X_test = train_test_split(df_train[numerical_features_not_null+[target]], test_size=0.1, random_state=0)
Y_train = X_train.pop(target)
Y_test = X_test.pop(target)

X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

((1314, 33), (1314,), (146, 33), (146,))

In [10]:
## feature scale

scaler = StandardScaler()

X_train = pd.DataFrame(scaler.fit_transform(X_train), index=X_train.index)
X_train.columns = numerical_features_not_null

X_test = pd.DataFrame(scaler.transform(X_test), index=X_test.index)
X_test.columns = numerical_features_not_null   

X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

((1314, 33), (1314,), (146, 33), (146,))

In [11]:
## experiment parameters 
K_folds = 5
results_experiments = {}

## 1. Grid search

### 1.1 original features

In [12]:
hyper_parameters = {'solver': ['svd', 'cholesky', 'lsqr', 'sag'],
                    'alpha':[0.01, 0.25, 0.5, 1.0, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024],
                    'fit_intercept': [True, False]
                   }

## all combinations
len(hyper_parameters['solver'])*len(hyper_parameters['alpha'])*len(hyper_parameters['fit_intercept'])

112

In [13]:
mse_score = make_scorer(mean_squared_error, greater_is_better=False)
lr = Ridge()
kf = KFold(n_splits=K_folds, shuffle=True, random_state=21)
grid_search = GridSearchCV(lr, hyper_parameters, scoring=mse_score, n_jobs=-1, cv=kf, 
                           refit=True)

grid_search

GridSearchCV(cv=KFold(n_splits=5, random_state=21, shuffle=True),
             estimator=Ridge(), n_jobs=-1,
             param_grid={'alpha': [0.01, 0.25, 0.5, 1.0, 2, 4, 8, 16, 32, 64,
                                   128, 256, 512, 1024],
                         'fit_intercept': [True, False],
                         'solver': ['svd', 'cholesky', 'lsqr', 'sag']},
             scoring=make_scorer(mean_squared_error, greater_is_better=False))

In [14]:
result = grid_search.fit(X_train.values, Y_train.values)

In [15]:
print(result.best_params_)
df_history = pd.DataFrame(result.cv_results_)[['mean_fit_time', 'param_alpha', 'param_fit_intercept', 'param_solver', 'mean_test_score']]\
.sort_values('mean_test_score', ascending=False)

time_fit = df_history['mean_fit_time'].sum()
print(time_fit)
df_history

{'alpha': 128, 'fit_intercept': True, 'solver': 'lsqr'}
0.7102257251739502


,mean_fit_time,param_alpha,param_fit_intercept,param_solver,mean_test_score
82,0.003204,128,True,lsqr,-1.392475e+09
81,0.001586,128,True,cholesky,-1.393060e+09
80,0.002825,128,True,svd,-1.393060e+09
83,0.013413,128,True,sag,-1.393321e+09
74,0.002814,64,True,lsqr,-1.397332e+09
...,...,...,...,...,...
13,0.001596,0.25,False,cholesky,-3.650954e+10
12,0.003134,0.25,False,svd,-3.650954e+10
7,0.024394,0.01,False,sag,-3.650970e+10
5,0.001646,0.01,False,cholesky,-3.651114e+10


In [16]:
## Train a final model
lr = Ridge(alpha=result.best_params_['alpha'], 
           fit_intercept= result.best_params_['fit_intercept'], 
           solver = result.best_params_['solver']).fit(X_train.values, Y_train.values)

y_predict_train = lr.predict(X_train.values)
y_predict_test = lr.predict(X_test.values)

metric_train = mean_squared_error(y_predict_train, Y_train.values)
metric_test = mean_squared_error(y_predict_test, Y_test.values)   
metric_mae_test = mean_absolute_error(y_predict_test, Y_test.values)   

print(metric_train, metric_test)
results_experiments['or_grid'] = [time_fit, result.best_params_['alpha'], result.best_params_['fit_intercept'], 
                                  result.best_params_['solver'], metric_train.copy(), 
                                  metric_test.copy(), metric_mae_test]

1194598333.3308284 1581843357.8697352


In [17]:
df_results = pd.DataFrame(results_experiments).T
df_results.columns = ['time', 'alpha', 'intercept', 'solver', 'train', 'test', 'mae-test']
df_results

,time,alpha,intercept,solver,train,test,mae-test
or_grid,0.710226,128,True,lsqr,1194598333.330828,1581843357.869735,21491.288767


### 1.2 polynomial features

In [18]:
num_features = 10
columns = list(X_train.columns)+[target]
corr_target = correlation_heatmap(pd.concat([X_train, pd.DataFrame(Y_train)], axis=1), 
                                 columns)
high_corr_features = list(corr_target[target].sort_values(ascending=False)[1:num_features+1].index)        
high_corr_features, len(high_corr_features)

(['OverallQual',
  'GrLivArea',
  'GarageCars',
  'GarageArea',
  'TotalBsmtSF',
  '1stFlrSF',
  'FullBath',
  'TotRmsAbvGrd',
  'YearBuilt',
  'YearRemodAdd'],
 10)

In [19]:
mse_score = make_scorer(mean_squared_error, greater_is_better=False)
lr = Ridge()
kf = KFold(n_splits=K_folds, shuffle=True, random_state=21)
grid_search = GridSearchCV(lr, hyper_parameters, scoring=mse_score, n_jobs=-1, cv=kf, 
                           refit=True)

grid_search

GridSearchCV(cv=KFold(n_splits=5, random_state=21, shuffle=True),
             estimator=Ridge(), n_jobs=-1,
             param_grid={'alpha': [0.01, 0.25, 0.5, 1.0, 2, 4, 8, 16, 32, 64,
                                   128, 256, 512, 1024],
                         'fit_intercept': [True, False],
                         'solver': ['svd', 'cholesky', 'lsqr', 'sag']},
             scoring=make_scorer(mean_squared_error, greater_is_better=False))

In [20]:
X_polyn_train =  polynomial_transformations(X_train, high_corr_features, degree=2)
X_polyn_test =  polynomial_transformations(X_test, high_corr_features, degree=2)

result = grid_search.fit(X_polyn_train.values, Y_train.values)

In [21]:
print(result.best_params_)
df_history = pd.DataFrame(result.cv_results_)[['mean_fit_time', 'param_alpha', 'param_fit_intercept', 'param_solver', 'mean_test_score']]\
.sort_values('mean_test_score', ascending=False)

time_fit = df_history['mean_fit_time'].sum()
print(time_fit)
df_history

{'alpha': 128, 'fit_intercept': True, 'solver': 'cholesky'}
9.203738641738891


,mean_fit_time,param_alpha,param_fit_intercept,param_solver,mean_test_score
81,0.001870,128,True,cholesky,-1.127000e+09
80,0.005911,128,True,svd,-1.127000e+09
82,0.003623,128,True,lsqr,-1.128358e+09
74,0.005078,64,True,lsqr,-1.139955e+09
73,0.002371,64,True,cholesky,-1.141058e+09
...,...,...,...,...,...
103,0.178139,512,False,sag,-4.720114e+09
108,0.005899,1024,False,svd,-6.597767e+09
109,0.001901,1024,False,cholesky,-6.597767e+09
110,0.002904,1024,False,lsqr,-6.599764e+09


In [22]:
## Train a final model
lr = Ridge(alpha=result.best_params_['alpha'], 
           fit_intercept= result.best_params_['fit_intercept'], 
           solver = result.best_params_['solver']).fit(X_polyn_train.values, Y_train.values)

y_predict_train = lr.predict(X_polyn_train.values)
y_predict_test = lr.predict(X_polyn_test.values)

metric_train = mean_squared_error(y_predict_train, Y_train.values)
metric_test = mean_squared_error(y_predict_test, Y_test.values)   
metric_mae_test = mean_absolute_error(y_predict_test, Y_test.values)   

print(metric_train, metric_test)
results_experiments['poly_grid'] = [time_fit, result.best_params_['alpha'], result.best_params_['fit_intercept'], 
                                    result.best_params_['solver'], metric_train.copy(), 
                                    metric_test.copy(), metric_mae_test]

867746947.6857089 937050590.9702797


In [23]:
df_results = pd.DataFrame(results_experiments).T
df_results.columns = ['time', 'alpha', 'intercept', 'solver', 'train', 'test', 'mae-test']
df_results

,time,alpha,intercept,solver,train,test,mae-test
or_grid,0.710226,128,True,lsqr,1194598333.330828,1581843357.869735,21491.288767
poly_grid,9.203739,128,True,cholesky,867746947.685709,937050590.97028,18500.485608


## 2. Random search

### 2.1. original features

In [24]:
hyper_parameters = {'solver': ['svd', 'cholesky', 'lsqr', 'sag'],
                    'alpha':loguniform(1e-5, 100),
                    'fit_intercept': [True, False]
                   }
n_iter = 50

In [25]:
mse_score = make_scorer(mean_squared_error, greater_is_better=False)
lr = Ridge()
kf = KFold(n_splits=K_folds, shuffle=True, random_state=21)
random_search = RandomizedSearchCV(lr, hyper_parameters, scoring=mse_score, n_jobs=-1, 
                                   cv=kf, refit=True, n_iter=100)

random_search

RandomizedSearchCV(cv=KFold(n_splits=5, random_state=21, shuffle=True),
                   estimator=Ridge(), n_iter=100, n_jobs=-1,
                   param_distributions={'alpha': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f959a237520>,
                                        'fit_intercept': [True, False],
                                        'solver': ['svd', 'cholesky', 'lsqr',
                                                   'sag']},
                   scoring=make_scorer(mean_squared_error, greater_is_better=False))

In [26]:
result = random_search.fit(X_train.values, Y_train.values)

In [27]:
print(result.best_params_)
df_history = pd.DataFrame(result.cv_results_)[['mean_fit_time', 'param_alpha', 'param_fit_intercept', 'param_solver', 'mean_test_score']]\
.sort_values('mean_test_score', ascending=False)

time_fit = df_history['mean_fit_time'].sum()
print(time_fit)
df_history

{'alpha': 25.92874250689745, 'fit_intercept': True, 'solver': 'lsqr'}
0.6443180561065674


,mean_fit_time,param_alpha,param_fit_intercept,param_solver,mean_test_score
77,0.003377,25.928743,True,lsqr,-1.406227e+09
74,0.016811,15.769274,True,sag,-1.410791e+09
98,0.003711,13.425807,True,lsqr,-1.410828e+09
16,0.003398,11.608945,True,lsqr,-1.411594e+09
10,0.002559,11.529566,True,lsqr,-1.411628e+09
...,...,...,...,...,...
82,0.001367,0.000285,False,cholesky,-3.651121e+10
15,0.002520,0.000199,False,svd,-3.651121e+10
58,0.003091,0.000092,False,svd,-3.651121e+10
71,0.001314,0.000086,False,cholesky,-3.651121e+10


In [28]:
## Train a final model
lr = Ridge(alpha=result.best_params_['alpha'], 
           fit_intercept= result.best_params_['fit_intercept'], 
           solver = result.best_params_['solver']).fit(X_train.values, Y_train.values)

y_predict_train = lr.predict(X_train.values)
y_predict_test = lr.predict(X_test.values)

metric_train = mean_squared_error(y_predict_train, Y_train.values)
metric_test = mean_squared_error(y_predict_test, Y_test.values)   
metric_mae_test = mean_absolute_error(y_predict_test, Y_test.values)   

print(metric_train, metric_test)
results_experiments['or_random'] = [time_fit, result.best_params_['alpha'], result.best_params_['fit_intercept'], 
                                      result.best_params_['solver'], metric_train.copy(), 
                                    metric_test.copy(), metric_mae_test]

1182020616.843114 1578978164.7964833


In [29]:
df_results = pd.DataFrame(results_experiments).T
df_results.columns = ['time', 'alpha', 'intercept', 'solver', 'train', 'test', 'mae-test']
df_results

,time,alpha,intercept,solver,train,test,mae-test
or_grid,0.710226,128,True,lsqr,1194598333.330828,1581843357.869735,21491.288767
poly_grid,9.203739,128,True,cholesky,867746947.685709,937050590.97028,18500.485608
or_random,0.644318,25.928743,True,lsqr,1182020616.843114,1578978164.796483,21904.385634


### 2.2 polynomial features

In [30]:
mse_score = make_scorer(mean_squared_error, greater_is_better=False)
lr = Ridge()
kf = KFold(n_splits=K_folds, shuffle=True, random_state=21)
random_search = RandomizedSearchCV(lr, hyper_parameters, scoring=mse_score, n_jobs=-1, cv=kf, 
                                   refit=True, n_iter=n_iter)

random_search

RandomizedSearchCV(cv=KFold(n_splits=5, random_state=21, shuffle=True),
                   estimator=Ridge(), n_iter=50, n_jobs=-1,
                   param_distributions={'alpha': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f959a237520>,
                                        'fit_intercept': [True, False],
                                        'solver': ['svd', 'cholesky', 'lsqr',
                                                   'sag']},
                   scoring=make_scorer(mean_squared_error, greater_is_better=False))

In [31]:
result = random_search.fit(X_polyn_train.values, Y_train.values)

In [32]:
print(result.best_params_)
df_history = pd.DataFrame(result.cv_results_)[['mean_fit_time', 'param_alpha', 'param_fit_intercept', 'param_solver', 'mean_test_score']]\
.sort_values('mean_test_score', ascending=False)

time_fit = df_history['mean_fit_time'].sum()
print(time_fit)
df_history

{'alpha': 82.30251237386368, 'fit_intercept': True, 'solver': 'svd'}
4.592164134979249


,mean_fit_time,param_alpha,param_fit_intercept,param_solver,mean_test_score
27,0.006020,82.302512,True,svd,-1.128725e+09
8,0.246393,78.424074,True,sag,-1.140484e+09
32,0.001740,47.139931,True,cholesky,-1.164115e+09
16,0.007094,43.172238,True,svd,-1.171947e+09
13,0.302931,13.039153,True,sag,-1.212894e+09
43,0.299444,0.000514,True,sag,-1.258050e+09
49,0.276288,0.700459,True,sag,-1.258791e+09
9,0.331320,0.011469,True,sag,-1.260521e+09
38,0.004897,13.188741,True,lsqr,-1.284432e+09
3,0.008053,8.394995,True,svd,-1.330801e+09


In [33]:
## Train a final model
lr = Ridge(alpha=result.best_params_['alpha'], 
           fit_intercept= result.best_params_['fit_intercept'], 
           solver = result.best_params_['solver']).fit(X_polyn_train.values, Y_train.values)

y_predict_train = lr.predict(X_polyn_train.values)
y_predict_test = lr.predict(X_polyn_test.values)

metric_train = mean_squared_error(y_predict_train, Y_train.values)
metric_test = mean_squared_error(y_predict_test, Y_test.values)   
metric_mae_test = mean_absolute_error(y_predict_test, Y_test.values)   

print(metric_train, metric_test)
results_experiments['poly_random'] = [time_fit, result.best_params_['alpha'], result.best_params_['fit_intercept'], 
                                      result.best_params_['solver'], metric_train.copy(), 
                                      metric_test.copy(), metric_mae_test]

851719251.1345408 941966771.0662988


In [34]:
df_results = pd.DataFrame(results_experiments).T
df_results.columns = ['time', 'alpha', 'intercept', 'solver', 'train', 'test', 'mae-test']
df_results

,time,alpha,intercept,solver,train,test,mae-test
or_grid,0.710226,128,True,lsqr,1194598333.330828,1581843357.869735,21491.288767
poly_grid,9.203739,128,True,cholesky,867746947.685709,937050590.97028,18500.485608
or_random,0.644318,25.928743,True,lsqr,1182020616.843114,1578978164.796483,21904.385634
poly_random,4.592164,82.302512,True,svd,851719251.134541,941966771.066299,18643.375882
